## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,76.30,86,57,26.84,broken clouds
1,1,Charlottesville,US,38.0293,-78.4767,39.20,93,90,2.04,fog
2,2,Port Lincoln,AU,-34.7333,135.8667,70.38,57,64,9.78,broken clouds
3,3,Barrow,US,71.2906,-156.7887,-16.60,76,90,6.91,haze
4,4,Kabo,CF,7.6994,18.6290,73.74,30,91,2.51,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Arraial Do Cabo,BR,-22.9661,-42.0278,76.30,86,57,26.84,broken clouds
5,5,Georgetown,MY,5.4112,100.3354,89.01,66,20,8.05,few clouds
6,6,Atuona,PF,-9.8000,-139.0333,79.02,75,3,16.73,clear sky
10,10,Asosa,ET,10.0667,34.5333,75.58,34,22,6.46,few clouds
16,16,Rikitea,PF,-23.1203,-134.9692,79.99,75,18,9.53,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                175
City                   175
Country                174
Lat                    175
Lng                    175
Max Temp               175
Humidity               175
Cloudiness             175
Wind Speed             175
Current Description    175
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,76.30,broken clouds,-22.9661,-42.0278,
5,Georgetown,MY,89.01,few clouds,5.4112,100.3354,
6,Atuona,PF,79.02,clear sky,-9.8000,-139.0333,
10,Asosa,ET,75.58,few clouds,10.0667,34.5333,
16,Rikitea,PF,79.99,light rain,-23.1203,-134.9692,
21,Kapaa,US,75.20,light rain,22.0752,-159.3190,
22,Hithadhoo,MV,81.16,overcast clouds,-0.6000,73.0833,
23,Mahajanga,MG,83.82,light rain,-15.7167,46.3167,
25,Progreso,MX,75.00,clear sky,21.2833,-89.6667,
26,Sechura,PE,75.36,overcast clouds,-5.5569,-80.8222,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Arraial Do Cabo,BR,76.30,broken clouds,-22.9661,-42.0278,Pousada Porto Praia
5,Georgetown,MY,89.01,few clouds,5.4112,100.3354,Cititel Penang
6,Atuona,PF,79.02,clear sky,-9.8000,-139.0333,Villa Enata
10,Asosa,ET,75.58,few clouds,10.0667,34.5333,Blendana Hotel Assosa
16,Rikitea,PF,79.99,light rain,-23.1203,-134.9692,Pension Maro'i
21,Kapaa,US,75.20,light rain,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
22,Hithadhoo,MV,81.16,overcast clouds,-0.6000,73.0833,Scoop Guest House
23,Mahajanga,MG,83.82,light rain,-15.7167,46.3167,Restaurant Greedy Coco Lodge Majunga
25,Progreso,MX,75.00,clear sky,21.2833,-89.6667,Playa Linda Hotel
26,Sechura,PE,75.36,overcast clouds,-5.5569,-80.8222,Hospedaje Costa Bella


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))